In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import time
from datetime import datetime
import os
import re

# --- WebDriver setup ---
service = Service(r"C:\Users\hp\Data Science ML\webscraping\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.maximize_window()

# --- Open form ---
driver.get("https://docs.google.com/forms/d/e/1FAIpQLSdUCd3UWQ3VOgeg0ZzNeT-xzNawU8AJ7Xidml-w1vhfBcvBWQ/viewform")

WebDriverWait(driver, 15).until(
    EC.presence_of_all_elements_located((By.XPATH, '//input | //textarea'))
)
time.sleep(2)

# --- Data to fill ---
data = {
    "name": "Rajeev",
    "contact": "1234567890",
    "email": "r@gmail.com",
    "address": "Delhi, India",
    "pincode": "110001",
    "dob": "01-02-2004",  
    "gender": "Male" # Will be sent as text
}

# --- Helper to safely fill fields ---
def safe_send_keys(element, text):
    """Scrolls to element and sends keys, handling interactivity issues."""
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", element)
    time.sleep(0.4)
    try:
        element.click()
        element.clear()
        element.send_keys(text)
    except ElementNotInteractableException:
        # Fallback to JavaScript execution if standard interaction fails
        driver.execute_script("arguments[0].value = arguments[1];", element, text)

# --- Find all question blocks ---
questions = driver.find_elements(By.XPATH, '//div[contains(@class,"geS5n")]')

for q in questions:
    try:
        raw_label = q.find_element(By.XPATH, './/div[@role="heading"]').text
        label = re.sub(r'^\d+\.\s*', '', raw_label).lower() 
    except (NoSuchElementException, ElementNotInteractableException):
        continue

    # --- Match label text and fill corresponding field ---
    if "name" in label:
        inp = q.find_element(By.XPATH, './/input')
        safe_send_keys(inp, data["name"])

    elif "contact" in label or "phone" in label:
        inp = q.find_element(By.XPATH, './/input')
        safe_send_keys(inp, data["contact"])

    elif "email" in label:
        inp = q.find_element(By.XPATH, './/input')
        safe_send_keys(inp, data["email"])

    elif "address" in label:
        try:
            inp = q.find_element(By.XPATH, './/textarea')
        except NoSuchElementException:
            inp = q.find_element(By.XPATH, './/input')
        safe_send_keys(inp, data["address"])

    elif "pin" in label or "zip" in label:
        inp = q.find_element(By.XPATH, './/input')
        safe_send_keys(inp, data["pincode"])

    elif "date" in label or "dob" in label or "birth" in label:
        inp = q.find_element(By.XPATH, './/input')
        inp.send_keys(data['dob'])

    elif "gender" in label or "sex" in label:
        try:
            # Find the single text input for the answer
            inp = q.find_element(By.XPATH, './/input') 
            safe_send_keys(inp, data["gender"])
            print(f"✅ Filled Gender: {data['gender']}")
        except NoSuchElementException:
            print("⚠️ Gender input field not found.")
        except Exception as e:
            print("⚠️ Could not fill gender:", e)

# --- Submit Form ---
# feed captcha-------------------------------------------------
code=driver.find_element(By.XPATH, '//div[@class="HoXoMd D1wxyf RjsPE"]//span[@class="M7eMe"]//b').text.strip()
driver.find_element(By.XPATH, '//input[@aria-labelledby="i37 i40"]').send_keys(code)
print(code)
# feed captcha-------------------------------------------------

try:
    submit_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@role="button"]//span[text()="Submit" or text()="Next"]/ancestor::div[@role="button"]'))
    )
    submit_btn.click()
    print("✅ Form submitted successfully.")
except Exception as e:
    print("⚠️ Could not find or click Submit button:", e)


# screenshot block-------------------------------------------------------------------------
shots_dir = os.path.join(os.getcwd(), "screenshots")
os.makedirs(shots_dir, exist_ok=True)
def timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def save_full_page_screenshot(driver, prefix="full"):
    """Save a full window screenshot to screenshots/ prefix_TIMESTAMP.png """
    filename = os.path.join(shots_dir, f"{prefix}_{timestamp()}.png")
    # Simple approach (works in most cases)
    driver.save_screenshot(filename)
    print(f"Saved full-page screenshot: {filename}")
    return filename

time.sleep(2)
save_full_page_screenshot(driver, prefix="after_submit")
print("Form submitted and final screenshot saved.")
# screenshot endblock ---------------------------------------------------------------------------

time.sleep(10)
driver.quit()

✅ Filled Gender: Male
GNFPYC
✅ Form submitted successfully.
Saved full-page screenshot: C:\Users\hp\Data Science ML\webscraping\assigment\screenshots\after_submit_20251026_131408.png
Form submitted and final screenshot saved.


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# --- WebDriver setup ---
service = Service(r"C:\Users\hp\Data Science ML\webscraping\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.maximize_window()

# --- Open Google Form ---
url = "https://docs.google.com/forms/d/e/1FAIpQLSdUCd3UWQ3VOgeg0ZzNeT-xzNawU8AJ7Xidml-w1vhfBcvBWQ/viewform"
driver.get(url)

# --- Wait for form fields to load ---
WebDriverWait(driver, 15).until(
    EC.presence_of_all_elements_located((By.XPATH, '//input | //textarea'))
)
time.sleep(2)

# --- Your data ---
data = {
    "Full Name": "Rajeev",
    "Contact Number": "9876543210",
    "Email ID": "r@gmail.com",
    "Full Address": "Delhi, India",
    "Pin Code": "110001",
    "Date of Birth": "01/01/2005",  # format: mm/dd/yyyy
    "Gender": "Male",
    "Type this code": "GNFPYC"  # you must type the actual captcha manually
}

# --- Fill fields dynamically ---
all_fields = driver.find_elements(By.XPATH, '//div[@role="listitem"]')

for field in all_fields:
    try:
        label = field.find_element(By.XPATH, './/div[@class="M7eMe"]').text.strip()
        if label in data:
            # Date field
            if "Date" in label:
                input_box = field.find_element(By.XPATH, './/input[@type="date"]')
                input_box.send_keys(data[label])
            
            # Normal text/number/email fields
            elif field.find_elements(By.XPATH, './/input[@type="text"]'):
                input_box = field.find_element(By.XPATH, './/input[@type="text"]')
                input_box.send_keys(data[label])
            
            # Textarea (e.g., Full Address)
            elif field.find_elements(By.XPATH, './/textarea'):
                input_box = field.find_element(By.XPATH, './/textarea')
                input_box.send_keys(data[label])
            
            time.sleep(0.5)
    except Exception as e:
        print(f"Skipping field due to: {e}")

time.sleep(3)
print("✅ All fields filled successfully (except CAPTCHA — fill it manually).")

# --- Wait for user to fill captcha manually then click submit ---
input("Press ENTER after you fill CAPTCHA manually...")

submit_btn = driver.find_element(By.XPATH, '//span[text()="Submit"]')
submit_btn.click()

print("Form submitted successfully!")
time.sleep(3)
driver.quit()


Skipping field due to: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="M7eMe"]"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x00007FF7091ECF55+75717]
	GetHandleVerifier [0x00007FF7091ECFB0+75808]
	(No symbol) [0x00007FF708FB8F9A]
	(No symbol) [0x00007FF70900F4C6]
	(No symbol) [0x00007FF70900F77C]
	(No symbol) [0x00007FF709001D7C]
	(No symbol) [0x00007FF7090373BF]
	(No symbol) [0x00007FF709001C46]
	(No symbol) [0x00007FF709037590]
	(No symbol) [0x00007FF70905F39C]
	(No symbol) [0x00007FF709037153]
	(No symbol) [0x00007FF709000421]
	(No symbol) [0x00007FF7090011B3]
	GetHandleVerifier [0x00007FF7094ED72D+3223453]
	GetHandleVerifier [0x00007FF7094E7CD2+3200322]
	GetHandleVerifier [0x00007FF709505B03+3322739]
	GetHandleVerifier [0x00007FF709206A2A+180890]
	GetHandleVerifier 

Press ENTER after you fill CAPTCHA manually... bvcx


Form submitted successfully!
